In [84]:
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from typing import Union
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
path = 'data/'
df = pd.read_csv(filepath_or_buffer=path+'cleaned_data.csv')
employee_df = pd.read_csv(filepath_or_buffer=path+'employee_data.csv')
# Let's take age and tenure as numerical features
df = df.merge(employee_df[['EmployeeID', 'Age', 'Tenure']], on='EmployeeID', how='left')

del employee_df


In [85]:
# Create Usage Vectors (Total Usage per BenefitSubtype)
usage_vectors = df.pivot_table(index='EmployeeID',
                                columns='BenefitSubType',
                                values='UsageFrequency',
                                aggfunc='sum').fillna(0)


# Normalize per employee
usage_vectors = usage_vectors.div(usage_vectors.sum(axis=1), axis=0)

# Create YearMonth column for temporal analysis
df['LastUsedDate'] = pd.to_datetime(df['LastUsedDate'])
df['YearMonth'] = df['LastUsedDate'].dt.to_period('M')

temporal_profiles = df.pivot_table(index='EmployeeID',
                                    columns='YearMonth',
                                    values='UsageFrequency',
                                    aggfunc='sum').fillna(0)

# Apply PCA to reduce noice
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
temporal_pca = pca.fit_transform(temporal_profiles)


In [86]:
# show numeric columns
df.select_dtypes(include=[np.number]).columns.tolist()

# Create satisfaction normalized by usage
df['SatisfactionNormalized_usage'] = df['SatisfactionScore'] / (df['UsageFrequency'] + 1e-5)


In [87]:
# Generate a new DataFrame with the temporal profiles
needed_cols = list(df.columns)
cols_to_del = ['Comments','LastUsedDate', 'BenefitType', 'BenefitSubType', 'YearMonth']

for col in list(df.columns):
    if (col in cols_to_del) or ('BenefitFlag_' in col):
        if col in needed_cols:
            needed_cols.remove(col)
df_analysis = df[needed_cols]

# remove duplicates from EmployeeID and BenefitID by grouping by EmployeeID and BenefitID 
df_analysis = df_analysis.groupby(by=["EmployeeID", "BenefitID"]).agg({
                                                                        'SatisfactionScore': 'mean',
                                                                        'SatisfactionNormalized_usage': 'mean',
                                                                        'UsageFrequency': 'mean',
                                                                        'BenefitCost': 'mean',
                                                                        'Gender_Female': 'max',
                                                                        'Gender_Male': 'max',
                                                                        'Gender_Non-Binary': 'max',
                                                                        'Department_Finance': 'max',
                                                                        'Department_HR': 'max',
                                                                        'Department_IT': 'max',
                                                                        'Department_Marketing': 'max',
                                                                        'Department_Sales': 'max',
                                                                        'Age_Gen_Boomer': 'max',
                                                                        'Age_Gen_Gen_X': 'max',
                                                                        'Age_Gen_Gen_Z': 'max',
                                                                        'Age_Gen_Millenial': 'max',
                                                                        'TenureGroups_ >25_years': 'max',
                                                                        'TenureGroups_16-25_years': 'max',
                                                                        'TenureGroups_5-15_years': 'max',
                                                                        'TenureGroups_<5_years': 'max',
                                                                        'Age' : 'mean',
                                                                        'Tenure' : 'mean'
                                                                    })


In [88]:
df_analysis.select_dtypes(include='number')

# Normalize the numerical columns except for 'EmployeeID' and 'BenefitID':
def normalize_data(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df_normalized = df.copy()
    for col in cols:
        if col in df_normalized.columns:
            df_normalized[col] = (df_normalized[col] - df_normalized[col].mean()) / df_normalized[col].std()
    return df_normalized    


# Get all numeric columns except 'EmployeeID' and 'BenefitID'
cols_to_normalize = [col for col in df_analysis.select_dtypes(include='number').columns if col not in ['EmployeeID', 'BenefitID']]

# Apply normalization only to the selected columns
df_analysis_normalized = normalize_data(df_analysis, cols_to_normalize)


In [89]:
# Function to generate temporal profiles using PCA
def apply_pca(df: pd.DataFrame, pca_components: Union[int, float] = 0.95) -> pd.DataFrame:
    pca = PCA(n_components=pca_components)
    pca_result = pca.fit_transform(df.select_dtypes(include='number'))
    pca_df = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(pca_result.shape[1])])
    return pca_df

# Function to find optimal number of clusters using the silhouette score
def find_optimal_clusters(df: pd.DataFrame, max_k: int = 10) -> int:
    from sklearn.metrics import silhouette_score
    best_k = 2
    best_score = -1
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        cluster_labels = kmeans.fit_predict(df)
        from sklearn.metrics import calinski_harabasz_score
        ch_score = calinski_harabasz_score(df, cluster_labels)
        from sklearn.metrics import davies_bouldin_score
        db_score = davies_bouldin_score(df, cluster_labels)
        silh_score = silhouette_score(df, cluster_labels)
        score = silh_score
        print(f'K={k}, Silhouette Score={score:.4f}, Calinski-Harabasz Score={ch_score:.4f}, Davies-Bouldin Score={db_score:.4f}')
        # Use silhouette score to determine the best number of clusters
        if score > best_score:
            best_score = score
            best_k = k
            print(f'New best K: {best_k} with score: {best_score:.4f}')
    return best_k

# Apply PCA to the normalized data
df_analysis_normalized_pca = apply_pca(df_analysis_normalized, pca_components=0.95)

# Find the optimal number of clusters
optimal_k = find_optimal_clusters(df_analysis_normalized_pca, max_k=8)

# Apply KMeans clustering
kmeans = KMeans(n_clusters=optimal_k, 
                max_iter=100,
                random_state=42,
                algorithm = 'elkan',
                n_init='auto')

# Add the cluster labels to the normalized DataFrame
df_analysis_normalized['Kmeans_seg'] = kmeans.fit_predict(df_analysis_normalized_pca)

# Store the centers of the clusters
kmeans_centers = kmeans.cluster_centers_
result = pd.concat([df_analysis_normalized.reset_index(), df_analysis_normalized_pca], axis=1)

K=2, Silhouette Score=0.1881, Calinski-Harabasz Score=1723.2377, Davies-Bouldin Score=2.0330
New best K: 2 with score: 0.1881
K=3, Silhouette Score=0.2126, Calinski-Harabasz Score=2060.0159, Davies-Bouldin Score=1.6190
New best K: 3 with score: 0.2126
K=3, Silhouette Score=0.2126, Calinski-Harabasz Score=2060.0159, Davies-Bouldin Score=1.6190
New best K: 3 with score: 0.2126
K=4, Silhouette Score=0.1921, Calinski-Harabasz Score=1840.8790, Davies-Bouldin Score=1.6143
K=4, Silhouette Score=0.1921, Calinski-Harabasz Score=1840.8790, Davies-Bouldin Score=1.6143
K=5, Silhouette Score=0.1903, Calinski-Harabasz Score=1724.6711, Davies-Bouldin Score=1.5087
K=5, Silhouette Score=0.1903, Calinski-Harabasz Score=1724.6711, Davies-Bouldin Score=1.5087
K=6, Silhouette Score=0.1833, Calinski-Harabasz Score=1572.1101, Davies-Bouldin Score=1.5287
K=6, Silhouette Score=0.1833, Calinski-Harabasz Score=1572.1101, Davies-Bouldin Score=1.5287
K=7, Silhouette Score=0.1805, Calinski-Harabasz Score=1502.6049,

In [90]:
# Visualize clusters in 3D with Plotly and show cluster centers
import plotly.graph_objects as go
# 3D scatter for points
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=result['PC1'], y=result['PC2'], z=result['PC3'],
    mode='markers',
    marker=dict(size=4, color=result['Kmeans_seg'], colorscale='Viridis', opacity=0.7),
    text=[f'Cluster {c}' for c in result['Kmeans_seg']],
    name='Data Points'
))
# 3D scatter for centers
fig.add_trace(go.Scatter3d(
    x=kmeans_centers[:,0], y=kmeans_centers[:,1], z=kmeans_centers[:,2],
    mode='markers',
    marker=dict(size=4, color='black', symbol='x'),
    name='Centers'
))
fig.update_layout(
    title='KMeans Clustering Results in 3D',
    scene = dict(xaxis_title='PC1', yaxis_title='PC2', zaxis_title='PC3'),
    legend=dict(x=0.8, y=0.9)
 )
fig.show()

# Visualize clusters in 2D with Plotly and show cluster centers
fig = px.scatter(result, x='PC1', y='PC2', color='Kmeans_seg', 
                 title='KMeans Clustering Results in 2D',
                 labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'})
# Add cluster centers
fig.add_trace(go.Scatter(
    x=kmeans_centers[:,0], y=kmeans_centers[:,1],
    mode='markers',
    marker=dict(size=15, color='black', symbol='x'),
    name='Centers'
))
fig.update_layout(
    title='KMeans Clustering Results in 2D',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    legend=dict(x=0.8, y=0.9)
 )
fig.show()

In [91]:
pd.merge(df_analysis, result[['EmployeeID', 'Kmeans_seg', 'BenefitID']], right_on=['EmployeeID','BenefitID'], left_on=['EmployeeID','BenefitID'])

,EmployeeID,BenefitID,SatisfactionScore,SatisfactionNormalized_usage,UsageFrequency,BenefitCost,Gender_Female,Gender_Male,Gender_Non-Binary,Department_Finance,Department_HR,Department_IT,Department_Marketing,Department_Sales,Age_Gen_Boomer,Age_Gen_Gen_X,Age_Gen_Gen_Z,Age_Gen_Millenial,TenureGroups_ >25_years,TenureGroups_16-25_years,TenureGroups_5-15_years,TenureGroups_<5_years,Age,Tenure,Kmeans_seg
0,2,6,1.0,0.200000,5.0,598.44,True,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,36.0,5.0,1
1,3,5,5.0,2.499988,2.0,75.00,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,56.0,6.0,0
2,3,23,4.0,0.999998,4.0,969.28,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,56.0,6.0,0
3,3,29,5.0,0.999998,5.0,475.00,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,56.0,6.0,0
4,4,28,3.0,1.499993,2.0,165.54,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,27.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7621,4998,29,3.0,0.999997,3.0,475.00,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,64.0,18.0,0
7622,4999,11,4.0,0.400000,10.0,928.75,False,True,False,True,False,False,False,False,False,True,False,False,True,False,False,False,54.0,30.0,0
7623,4999,18,1.0,100000.000000,0.0,743.01,False,True,False,True,False,False,False,False,False,True,False,False,True,False,False,False,54.0,30.0,0
7624,5000,19,5.0,0.555555,9.0,343.73,False,True,False,False,False,False,False,True,True,False,False,False,True,False,False,False,63.0,32.0,0


In [92]:
# Profile clussters by usage patterns and demographics
df_analysis_clustered = pd.merge(df_analysis, result[['EmployeeID', 'Kmeans_seg', 'BenefitID']], right_on=['EmployeeID','BenefitID'], left_on=['EmployeeID','BenefitID'])

# Aggregate the data to profile clusters
df_cluster_profile = df_analysis_clustered.groupby("Kmeans_seg").agg({
    'SatisfactionScore': 'mean',
    'UsageFrequency': 'mean',
    'BenefitCost': 'mean',
    'SatisfactionNormalized_usage': 'mean',
    'Age': 'mean',
    'Tenure' : 'mean',

    
    # Gender distribution
    'Gender_Female': 'sum',
    'Gender_Male': 'sum',
    'Gender_Non-Binary': 'sum',

    # Department distribution
    'Department_Finance': 'sum',
    'Department_HR': 'sum',
    'Department_IT': 'sum',
    'Department_Marketing': 'sum',
    'Department_Sales': 'sum',

    # Age groups
    'Age_Gen_Boomer': 'sum',
    'Age_Gen_Gen_X': 'sum',
    'Age_Gen_Gen_Z': 'sum',
    'Age_Gen_Millenial': 'sum',

    # Tenure groups
    'TenureGroups_ >25_years': 'sum',
    'TenureGroups_16-25_years': 'sum',
    'TenureGroups_5-15_years': 'sum',
    'TenureGroups_<5_years': 'sum'
})


In [93]:
import plotly.express as px
cluster_sizes = df_analysis_clustered['Kmeans_seg'].value_counts().sort_index().reset_index()
cluster_sizes.columns = ['Cluster', 'Number of Employees']
fig = px.bar(cluster_sizes, x='Cluster', y='Number of Employees',
             labels={'Cluster': 'Cluster', 'Number of Employees': 'Number of Employees'},
             title='Cluster Sizes',
             text='Number of Employees',
             height=600)
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Cluster', yaxis_title='Number of Employees')
fig.show()

In [ ]:
df_analysis_clustered_benefits = pd.merge(df_analysis_clustered,df[['EmployeeID', 'BenefitID','BenefitType', 'BenefitSubType', 'YearMonth']].drop_duplicates(), on=['EmployeeID', 'BenefitID'], how='inner')
usage_cluster_pct = df_analysis_clustered_benefits.groupby(['Kmeans_seg', 'BenefitType'])['UsageFrequency'].sum().unstack(fill_value=0)
satisfaction_matrix = df_analysis_clustered_benefits.groupby(['Kmeans_seg', 'BenefitType'])['SatisfactionScore'].mean().unstack(fill_value=0)

# Plotly heatmap for Benefit Usage % per Cluster (interactive)
import plotly.express as px
usage_pct = usage_cluster_pct.div(usage_cluster_pct.sum(axis=1), axis=0) * 100
usage_pct_long = usage_pct.reset_index().melt(id_vars='Kmeans_seg', var_name='BenefitType', value_name='UsagePercent')
fig = px.imshow(
    usage_pct.values, 
    labels=dict(x="Benefit Type", y="Cluster", color="Usage %"),
    x=usage_pct.columns,
    y=usage_pct.index,
    color_continuous_scale='YlGnBu',
    aspect='auto',
    text_auto='.1f',
    title="Benefit Usage % per Cluster"
 )
fig.update_xaxes(title_text="Benefit Type", tickangle=270)
fig.update_yaxes(title_text="Cluster", tickmode='linear', dtick=1)
fig.update_layout(height=500)
fig.show()

# Plotly heatmap for Avg Satisfaction Score per BenefitType and Cluster (interactive)
fig = px.imshow(
    satisfaction_matrix.values, 
    labels=dict(x="Benefit Type", y="Cluster", color="Avg Satisfaction Score"),
    x=satisfaction_matrix.columns,
    y=satisfaction_matrix.index,
    color_continuous_scale='YlGnBu',
    aspect='auto',
    text_auto='.2f',
    title="Avg Satisfaction Score per Benefit Type and Cluster"
 )
fig.update_xaxes(title_text="Benefit Type", tickangle=270)
fig.update_yaxes(title_text="Cluster", tickmode='linear', dtick=1)
fig.update_layout(height=500)
fig.show()

In [108]:
cluster_sizes = df_analysis_clustered_benefits[['EmployeeID', 'Kmeans_seg']].drop_duplicates().groupby('Kmeans_seg').size()
benefit_cluster_counts = (
    df_analysis_clustered_benefits
    .groupby(['BenefitType', 'Kmeans_seg'])['EmployeeID']
    .nunique()
    .unstack(fill_value=0)
)
benefit_usage_rate = benefit_cluster_counts.div(cluster_sizes, axis=1) * 100  # in %

#plot
def plot_benefit_usage_rate(benefit_usage_rate: pd.DataFrame, demo: str = 'Cluster'):
    fig = px.bar(benefit_usage_rate.reset_index().melt(id_vars='BenefitType', var_name=demo, value_name='Percent'),
                  x=demo,
                  y='Percent',
                  color='BenefitType',
                  barmode='group',
                  title=f'Benefit Usage Rate by {demo}',
                  labels={'Percent': 'Percent of Benefit Type'},
                  text='Percent',
                  height=600)
    fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig.update_layout(xaxis_title=demo, yaxis_title='Percent of Benefit Type', yaxis_tickformat='.1f')  # Ensure y-axis is formatted as percentage
    fig.show()

plot_benefit_usage_rate(benefit_usage_rate, demo='Kmeans_seg')

In [109]:
satisfaction_by_cluster = (
    df_analysis_clustered_benefits
    .groupby(['BenefitType', 'Kmeans_seg'])['SatisfactionScore']
    .mean()
    .unstack(fill_value=0)
)
def plot_satisfaction_score(satisfaction_df: pd.DataFrame, demo: str = 'Cluster'):
    fig = px.bar(
        satisfaction_df.reset_index().melt(id_vars='BenefitType', var_name=demo, value_name='AvgSatisfaction'),
        x=demo,
        y='AvgSatisfaction',
        color='BenefitType',
        barmode='group',
        title=f'Average Satisfaction Score by {demo}',
        labels={'AvgSatisfaction': 'Avg. Satisfaction Score'},
        text='AvgSatisfaction',
        height=600
    )
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(xaxis_title=demo, yaxis_title='Avg. Satisfaction Score')
    fig.show()
plot_satisfaction_score(satisfaction_by_cluster, demo='Kmeans_seg')

In [110]:
weighted_satisfaction = (
    df_analysis_clustered_benefits
    .assign(weighted_score=lambda df: df['SatisfactionScore'] * df['UsageFrequency'])
    .groupby(['BenefitType', 'Kmeans_seg'])
    .apply(lambda g: g['weighted_score'].sum() / g['UsageFrequency'].sum())
    .unstack(fill_value=0)
)

def plot_weighted_satisfaction(weighted_satisfaction_df: pd.DataFrame, demo: str = 'Cluster'):
    fig = px.bar(
        weighted_satisfaction_df.reset_index().melt(id_vars='BenefitType', var_name=demo, value_name='WeightedSatisfaction'),
        x=demo,
        y='WeightedSatisfaction',
        color='BenefitType',
        barmode='group',
        title=f'Weighted Satisfaction Score by {demo}',
        labels={'WeightedSatisfaction': 'Weighted Satisfaction Score'},
        text='WeightedSatisfaction',
        height=600
    )
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fig.update_layout(xaxis_title=demo, yaxis_title='Weighted Satisfaction Score')
    fig.show()
plot_weighted_satisfaction(weighted_satisfaction, demo='Kmeans_seg')

In [ ]:
# Demographic exploration of clusters with Plotly Express
demographic_features = {
    'Gender': ['Gender_Female', 'Gender_Male', 'Gender_Non-Binary'],
    'Department': ['Department_Finance', 'Department_HR', 'Department_IT', 'Department_Marketing', 'Department_Sales'],
    'Age Group': ['Age_Gen_Boomer', 'Age_Gen_Gen_X', 'Age_Gen_Gen_Z', 'Age_Gen_Millenial'],
    'Tenure Group': ['TenureGroups_ >25_years', 'TenureGroups_16-25_years', 'TenureGroups_5-15_years', 'TenureGroups_<5_years']
}

for demo, cols in demographic_features.items():
    demo_counts = df_analysis_clustered.groupby('Kmeans_seg')[cols].sum()
    demo_counts_percent = demo_counts.div(demo_counts.sum(axis=1), axis=0) * 100
    demo_long = demo_counts_percent.reset_index().melt(id_vars='Kmeans_seg', var_name=demo, value_name='Percent')
    fig = px.bar(
        demo_long, 
        x='Kmeans_seg', 
        y='Percent', 
        color=demo, 
        barmode='stack', 
        labels={'Kmeans_seg': 'Cluster', 'Percent': 'Percent'},
        title=f'{demo} Distribution by Cluster',
        height=400
    )
    fig.update_layout(xaxis_title='Cluster', yaxis_title='Percent', legend_title=demo)
    fig.update_xaxes(tickmode='linear', dtick=1)  # Ensure integer ticks on x-axis
    fig.show()

In [116]:
age_columns = ['Age_Gen_Boomer', 'Age_Gen_Gen_X', 'Age_Gen_Gen_Z', 'Age_Gen_Millenial']

df_analysis_clustered_benefits['AgeGroup'] = df_analysis_clustered_benefits[age_columns].idxmax(axis=1)
df_analysis_clustered_benefits['AgeGroup'] = df_analysis_clustered_benefits['AgeGroup'].str.replace('Age_Gen_', '')
df_analysis_clustered_benefits.groupby(['AgeGroup', 'BenefitType'])['UsageFrequency'].mean().unstack(fill_value=0)

#sum rows to get the total usage frequency per age group
age_usage = df_analysis_clustered_benefits.groupby('AgeGroup')['UsageFrequency'].sum().reset_index()
age_usage['Percent'] = age_usage['UsageFrequency'] / age_usage['UsageFrequency'].sum() * 100

fig = px.bar(
    age_usage, 
    x='AgeGroup', 
    y='Percent', 
    color='AgeGroup',
    title='Usage Frequency by Age Group',
    labels={'Percent': 'Usage Frequency (%)'},
    text='Percent',
    height=400
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(xaxis_title='Age Group', yaxis_title='Usage Frequency (%)')
fig.update_xaxes(tickmode='linear', dtick=1)  # Ensure integer ticks on x-axis
fig.show()


In [118]:
df_analysis_clustered_benefits.groupby(['AgeGroup', 'BenefitType'])['UsageFrequency'].mean().unstack(fill_value=0)

BenefitType,Cell Phone Allowance,Childcare,Commuter Benefits,Flexible Spending Account,Gym Membership,Health Insurance,Life Insurance,Professional Development,Retirement Plan,Technology Stipend,Tuition Reimbursement,Wellness Programs
AgeGroup,,,,,,,,,,,,
Boomer,4.172727,2.388889,3.721311,3.769231,3.344340,3.251111,3.211538,3.732143,3.640000,3.547170,3.055556,2.949153
Gen_X,3.077558,3.199830,3.009524,3.472727,3.163895,3.487981,3.563397,3.396552,3.443262,3.267327,3.510813,2.937500
Gen_Z,3.423077,2.000000,2.969697,3.411765,3.524752,3.387931,3.433962,3.304348,3.714286,3.158537,3.263780,2.944444
Millenial,3.764706,2.830918,3.573529,2.865385,3.348839,3.124317,3.532374,3.442308,3.537127,3.691176,3.060995,3.475000


# Findings
- We segmented into 3 main clusters, the main variables we used was demographics variables, satisfaction score, usage frequency, and benefit cost.
- Cluster 0: Group of employees with moderate satisfaction level. Observe that This group reports high satisfaction with benefits like Flexible Spending Accounts, despite relatively low usage — possibly indicating that while valuable, these benefits serve niche needs.
- Cluster 1: Group of employees with low satisfaction level. Despite low satisfaction levels, this group makes frequent use of the Retirement Plan.
- Cluster 2: Group of employees with overall very high satisfaction. Apart from the Insurance and Retirement benefits, Childcare, Gym Membersip is a popular benefit among this group.
- Demographic analysis shows that gender and department distribution is quite balanced across clusters, with some variations in age and tenure groups.
- Age seems to be a significant factor for clustering, Gen X and Boomers tend to be more likely in cluster 0, where as Gen-Z or Millenial to be in that cluster is very unlikely. Gen X and Boomers make 92% of Cluster 0. On the other hand, Cluster 1 consists of over 60% of Millenials. And the last cluster seems to consist mostly from Millenials and Gen X. Most of the Gen Z belonged to Cluster 1.
- Another possibly significant factor is tenure. Those employees with tenure smaller than 15, belong to cluster 1 (Over 95% of employees that are in Cluster 1 have tenure 0-15). Those that have tenure more than 25 year are in Cluster 0

- Boomers show higher usage of Commuter Benefits, Flexible Spendiang Acocunt, Cell phone Allowance and Professional Development
- Gen X uses Tuition Reimbursment and Childcare slighly more than others.
- Millennials show higher usage of Wellness Programs than any of the other groups.
- Gen Z has low overall benefit usage but is slightly more engaged with Gym and Retirement. Very low engagment in Chilcare.

- Interestingly, benefits favored by Boomers (e.g., Commuter Benefits, Flexible Spending) are also common in Cluster 0 — where Boomers dominate. This suggests a potential age-related alignment within clusters
